In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import torch.nn.functional as F
import cv2 as cv
import pytorch_lightning as pl

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.optim as optim  
import torch



from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, validation_curve

#import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib.colors import ListedColormap

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
# Convolutional Nueral Network
import torchmetrics
class basicCNNModel(pl.LightningModule):
    def __init__(self):
        super(basicCNNModel, self).__init__()
        kernel = 3
        stride = 1
        padding = 1
        self.layer1 = nn.Sequential(nn.Conv2d(3, out_channels=32, kernel_size=kernel, stride=stride, padding=padding),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2,stride=2),
                                   nn.Dropout(0.2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, out_channels=64, kernel_size=kernel, stride=stride, padding=padding),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2,stride=2),
                                   nn.Dropout(0.2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, out_channels=128, kernel_size=kernel, stride=stride, padding=padding),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2,stride=2),
                                   nn.Dropout(0.2))                                    
        
        self.fc1 = nn.Linear(8192,128)
        self.fc2 = nn.Linear(128,10)
        
        
        # initializing metrics
        # Accuracy
        self.train_accuracy = torchmetrics.Accuracy().to(torch.device("cuda", 0))
        self.valid_accuracy = torchmetrics.Accuracy(compute_on_step=False).to(torch.device("cuda", 0))
        self.test_accuracy = torchmetrics.Accuracy().to(torch.device("cuda", 0))
        
        # AUROC
        self.train_auroc = torchmetrics.AUROC(num_classes=10).to(torch.device("cuda", 0))
        self.valid_auroc = torchmetrics.AUROC(num_classes=10).to(torch.device("cuda", 0))
        self.valid_auroc = torchmetrics.AUROC(num_classes=10).to(torch.device("cuda", 0))
    
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = torch.flatten(out,1)
        out = nn.ReLU()(self.fc1(out))
        out = self.fc2(out)
        return out

    #This contains the manupulation on data that needs to be done only once such as downloading it
    def prepare_data(self):
        self.data_dir = "../input/tomatoleaf/tomato"
        #transforms
        self.train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(64),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()]) 
        self.test_transform = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(64),
                                     transforms.ToTensor()])
        # split into train and validation
        self.train_dataset, self.val_dataset = random_split(datasets.ImageFolder(self.data_dir+"/train",
                                                                                 transform = self.train_transform), [7500,2500])
        self.test_dataset = datasets.ImageFolder(self.data_dir+"/val", transform = self.test_transform)
        
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=32)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=32)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=32)
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters())
    
    def training_step(self,batch,batch_idx):
        #extracting input and output from the batch
        x,labels = batch
        #doing a forward pass
        preds = self.forward(x)
        #CE Loss is a combination of Log Softmax and NLL loss
        loss = nn.CrossEntropyLoss()(preds,labels)
        _,actual_preds = torch.max(preds,1)
        acc = self.train_accuracy(actual_preds,labels)
        self.log("training_loss",loss,on_epoch=True, prog_bar=True)#, logger=True)
        self.log("train_accuracy",acc,on_epoch=True, prog_bar=True)#, logger=True)

       
        output = { 
              #REQUIRED: It ie required for us to return "loss"
            "loss":loss,
            "train_accuracy": acc,
            "train_preds":actual_preds,
            "train_targets":labels,
        }
       
        return output
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack(
            tuple(
                output["loss"]
                for output in outputs
            )
        ).mean()

        avg_acc =  self.train_accuracy.compute()

        preds = []
        target = []
        for output in outputs :
          for idx in range(len(output["train_preds"])):
            preds.append(output["train_preds"][idx])
            target.append(output["train_targets"][idx])
            
        
        #auroc_score = self.train_auroc(torch.tensor(preds),torch.tensor(labels))
        # print("Avg Training Accuracy for this epoch : ", avg_acc)
        #print("Avg Training AUROC for this epoch : ", auroc)
        
        return     
    
    def validation_step(self, batch, batch_idx):
        preds = self.eval().forward(batch[0])
        loss = nn.CrossEntropyLoss()(preds,batch[1])

        _,actual_pred = torch.max(preds,1)
        x,labels = batch
        acc = self.valid_accuracy(actual_pred,labels)
        
        
        return {"batch_val_loss": loss , "val_accuracy":acc , "preds":actual_pred,"labels":labels}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack(
            tuple(
                output["batch_val_loss"]
                for output in outputs
            )
        ).mean()

        avg_acc =  self.valid_accuracy.compute()

        preds = []
        target = []
        for output in outputs :
          for idx in range(len(output["preds"])):
            preds.append(output["preds"][idx])
            target.append(output["labels"][idx])

        #auroc = self.valid_auroc(torch.tensor(target),torch.tensor(preds))
        #print("Avg Validation Accuracy for this epoch : ", avg_acc)
        #print("Avg Validation AUROC for this epoch : ", auroc)
        
        self.log("avg_val_loss",avg_loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("avg_val_acc",avg_acc, on_epoch=True, prog_bar=True, logger=True)
        
        
        
        return {
            "val_loss": avg_loss,
            #"val_auroc": auroc,
            
        }
    
    def test_step(self,batch,batch_idx):
        x,labels = batch
        preds = self.eval().forward(x)
        loss = nn.CrossEntropyLoss()(preds,labels)

        _,actual_pred = torch.max(preds,1)
        acc = self.test_accuracy(actual_pred,labels)
        test_metrics = {
                        "test_loss": loss,
                        "test_accuracy" : acc,
                       }
        self.log_dict(test_metrics)
        
    
    
trainer = pl.Trainer(gpus=1, max_epochs=20)  

model = basicCNNModel()
trainer.fit(model)
print(model)

trainer.test(model)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

basicCNNModel(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
  )
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (train_accuracy): Accuracy()
  (valid_accuracy): Accuracy()
  (test_accuracy): Accuracy()
  (train_auroc): AUROC()
  (valid_auroc): A

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8600000143051147, 'test_loss': 0.3963444232940674}
--------------------------------------------------------------------------------


[{'test_loss': 0.3963444232940674, 'test_accuracy': 0.8600000143051147}]